In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# 데이터셋 로딩
dataset = load_dataset("lmqg/qg_koquad")

In [ ]:
dataset

In [ ]:
# gemma 모델 훈련 프롬프트 만드는 함수 
# => 훈련모델 종류 및 데이터셋 종류에 따라 커스터마이징 해야 함.
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['paragraph'])):
 
        question = example['paragraph'][0] + '\n\nQ:' + example['question'][0]+'\nA:'
        answer = example['sentence'][0]
        
        prompt_list.append(r"""<bos><start_of_turn>user
                            {}<end_of_turn>
                            <start_of_turn>model
                            {}<end_of_turn><eos>""".format(question, answer))
    return prompt_list


In [ ]:
train_data = dataset['train']
print(generate_prompt(train_data[:10]))

In [ ]:
# QLoRA 설정
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# 모델 로딩
BASE_MODEL = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

In [ ]:
# Trainer 설정
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        #max_steps=3000,
        per_device_train_batch_size=2,  # batch_size=1 일때 10G, 2면 16G
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

In [ ]:
# 훈련시작
trainer.train()

In [ ]:
# loara_adapter 저장
ADAPTER_MODEL = "lora_adapter"
trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
# 파일저장 확인
!ls -alh lora_adapter

In [ ]:
# 원본모델과 lora_adapter 합쳐서 1개 모델을 만듬.
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-qa-ko')

In [ ]:
# 평가
